# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from scipy.stats import linregress
from scipy.stats import stats
import json

# Import API key
from api_keys import g_key

from citipy import citipy

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = os.path.join('output_data', 'cities.csv')
vacation_df = pd.read_csv(csv_file)
vacation_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Butaritari,1,KI,1604382432,76,3.07,172.79,80.82,16.89
1,1,Khao Yoi,20,TH,1604382433,62,13.24,99.82,87.80,11.41
2,2,Tiksi,100,RU,1604382435,93,71.69,128.87,5.79,2.42
3,3,Avarua,100,CK,1604382436,78,-21.21,-159.78,75.20,21.23
4,4,Rikitea,44,PF,1604382437,77,-23.12,-134.97,72.14,17.85
...,...,...,...,...,...,...,...,...,...,...
521,521,Setermoen,75,NO,1604383127,87,68.86,18.35,44.60,29.97
522,522,Sakaiminato,75,JP,1604383128,51,35.55,133.23,62.60,20.80
523,523,Tynda,2,RU,1604383130,88,55.17,124.72,8.73,5.73
524,524,Seoul,20,KR,1604382871,31,37.57,126.98,46.40,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = vacation_df[["Lat", "Lng"]]
humidity = vacation_df["Humidity"]

fig = gmaps.figure(center = (0, 0), zoom_level=1.75)
heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity, 
    dissipating=False,
    max_intensity=100,
    point_radius = 1
)
fig.add_layer(heat_layer)

In [4]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_df.drop(vacation_df[vacation_df["Max Temp"] > 80].index, inplace = True)
vacation_df.drop(vacation_df[vacation_df["Max Temp"] < 70].index, inplace = True)
vacation_df.drop(vacation_df[vacation_df["Cloudiness"] == 0].index, inplace = True)
vacation_df.drop(vacation_df[vacation_df["Wind Speed"] > 10].index, inplace = True)
vacation_df.drop(vacation_df[vacation_df["Humidity"] > 50].index, inplace = True)
vacation_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
35,35,Airai,84,TL,1604382480,47,-8.93,125.41,79.66,3.42
193,193,Bara,48,NG,1604382693,37,10.37,10.73,74.08,6.67
196,196,Baghdad,40,IQ,1604382677,38,33.34,44.40,75.20,4.70
287,287,Arlit,20,NE,1604382819,24,18.74,7.39,71.06,7.00
326,326,Taoudenni,99,ML,1604382871,23,22.68,-3.98,73.96,5.82
360,360,Gombe,21,NG,1604382917,37,10.29,11.17,73.80,8.16
436,436,Las Vegas,40,US,1604382931,23,36.17,-115.14,73.40,6.93
494,494,Ambalavao,13,MG,1604383093,33,-21.83,46.93,76.62,4.45
501,501,Viedma,100,AR,1604383102,47,-40.81,-63.00,70.00,5.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
vacation_df['Hotel'] = ""
vacation_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel
35,35,Airai,84,TL,1604382480,47,-8.93,125.41,79.66,3.42,
193,193,Bara,48,NG,1604382693,37,10.37,10.73,74.08,6.67,
196,196,Baghdad,40,IQ,1604382677,38,33.34,44.40,75.20,4.70,
287,287,Arlit,20,NE,1604382819,24,18.74,7.39,71.06,7.00,
326,326,Taoudenni,99,ML,1604382871,23,22.68,-3.98,73.96,5.82,
360,360,Gombe,21,NG,1604382917,37,10.29,11.17,73.80,8.16,
436,436,Las Vegas,40,US,1604382931,23,36.17,-115.14,73.40,6.93,
494,494,Ambalavao,13,MG,1604383093,33,-21.83,46.93,76.62,4.45,
501,501,Viedma,100,AR,1604383102,47,-40.81,-63.00,70.00,5.99,


In [8]:
for index, row in vacation_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
        "keyword": "hotel",
        "radius": 25000,
        "key": g_key,
        }
        lat = row['Lat']
        lng = row['Lng']
        params['location'] = f"{lat}, {lng}"
        hotel_data = requests.get(base_url, params=params).json()
        vacation_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
       
    except IndexError: 
        vacation_df.loc[index, "Hotel Name"] = "NaN"
vacation_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel,Hotel Name
35,35,Airai,84,TL,1604382480,47,-8.93,125.41,79.66,3.42,,Pousada Alecrim Namrau
193,193,Bara,48,NG,1604382693,37,10.37,10.73,74.08,6.67,,NaN
196,196,Baghdad,40,IQ,1604382677,38,33.34,44.40,75.20,4.70,,Babylon Rotana
287,287,Arlit,20,NE,1604382819,24,18.74,7.39,71.06,7.00,,Hôtel Telwa Bungalow
326,326,Taoudenni,99,ML,1604382871,23,22.68,-3.98,73.96,5.82,,NaN
360,360,Gombe,21,NG,1604382917,37,10.29,11.17,73.80,8.16,,LELAH VISITORS SUITE
436,436,Las Vegas,40,US,1604382931,23,36.17,-115.14,73.40,6.93,,Caesars Palace
494,494,Ambalavao,13,MG,1604383093,33,-21.83,46.93,76.62,4.45,,BETSILEO COUNTRY LODGE
501,501,Viedma,100,AR,1604383102,47,-40.81,-63.00,70.00,5.99,,Hotel Austral Viedma


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
